## 대화체 반말로 바꾸기

- https://huggingface.co/squarelike/korean-style-converter-6b

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
import torch
repo = "squarelike/korean-style-converter-6b"
model = AutoModelForCausalLM.from_pretrained(
        repo,
        # load_in_4bit=True,
        # device_map='auto',
        cache_dir='./cache'
)
tokenizer = AutoTokenizer.from_pretrained(repo)

model.safetensors.index.json: 100%|██████████| 23.9k/23.9k [00:00<00:00, 59.7MB/s]
model-00001-of-00003.safetensors: 100%|██████████| 4.96G/4.96G [03:11<00:00, 25.9MB/s]
model-00002-of-00003.safetensors: 100%|██████████| 4.93G/4.93G [03:22<00:00, 24.4MB/s]
model-00003-of-00003.safetensors: 100%|██████████| 2.46G/2.46G [01:59<00:00, 20.5MB/s]
generation_config.json: 100%|██████████| 137/137 [00:00<00:00, 657kB/s]
tokenizer_config.json: 100%|██████████| 9.56k/9.56k [00:00<00:00, 24.0MB/s]
tokenizer.json: 100%|██████████| 4.28M/4.28M [00:01<00:00, 3.86MB/s]
special_tokens_map.json: 100%|██████████| 573/573 [00:00<00:00, 2.88MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [66]:
# 데이터 불러오기

import pandas as pd

data_df = pd.read_csv('../data/train/session_2_age_20.csv')

In [67]:
data_df

,id,session_dialog,session_persona
0,K5-03984-CL74442-CP22286-13-04-S2.json,"['반가워요~~ 저는 20대 여자입니다. 저는 아메리카노를 자주 마셔요.', '오 ...","['나는 20대 여자이다. 나는 아메리카노를 자주 마신다.', '나는 뉴질랜드로 어..."
1,K5-03984-CL74442-CP22286-13-04-S2.json,"['아 반갑습니다. 저는 20대 여자이고요. 도서관 사서로 일하고 있어요.', '아...",['나는 20대 여자이다. 나는 도서관 사서이다.']
2,K5-03994-CL74403-CP11608-13-05-S2.json,"['안녕하세요! 저는 부천에 사는 10대 여자입니다!', '만나서 반가워요 ㅎㅎ 같...","['나는 10대 여성이다. 내가 사는 곳은 부천이다.', '나는 요즘 즐겁다. 나는..."
3,K5-03994-CL74403-CP11608-13-05-S2.json,"['반가워요! ㅎㅎ 저도 10대 여성이에요!', '저는 요즘 진학 문제때문에 너무 ...","['나는 10대 여자이다.', '나는 진학문제 때문에 고민이다. 나는 진학문제로 아..."
4,K5-03997-CL74070-CP11701-04-01-S2.json,"['반가워요 ㅋㅋ 저는20대 남성인데 그쪽은요??', 'ㅋㅋ 저는 지금 자기주장이 ...","['나는 20대 남자이다.', '나는 자기주장이 강하다. 나는 전기정보공학부 부전공..."
...,...,...,...
41311,K2-00002-CL21649-CP31111-08-09-S2.json,['네네 거의 한달만에 연락드리네요 저는 학생들이랑 수업하고 요즘 수행평가기간이라 ...,['나는 학생들 이과 수업하고 요즘 수행평가기간이라 아이들 수행평가를 지도해주면서 ...
41312,K2-00014-CL20283-CP21216-03-06-S2.json,"['아니 하루뒤에나 연락드렸네요... 일이 좀 있었어요.', '아니 어제 장을 보러...","['나는 볼 일이 있었다.', '나는 차 사고가 났다.', '나는 난기류를 만났을 ..."
41313,K2-00014-CL20283-CP21216-03-06-S2.json,"['무슨 일이신데요?', '네??? 그게 무슨 말씀인가요? 병원 가야하나요?', '...","['나는 비행기 트라우마가 있다. 나는 성인이 되고 나서 안 타봤다.', '나는 난..."
41314,K2-00020-CL13719-CP12624-03-09-S2.json,"['안녕하세요~ 2주만이네요~ 요즘도 요리 관련 동아리 활동 잘 하고 계신가요?',...","['나는 운전면허를 따고 싶다', '나는 대중교통에서 탈출하고 자가용을 이용하고 싶..."


In [68]:
# data_df에서 짝수번째 행과 홀수번째 행으로 분리
even_data_df = data_df.iloc[::2, :]
odd_data_df = data_df.iloc[1::2, :]

len(even_data_df), len(odd_data_df)

(20658, 20658)

In [69]:
# 각각에서 25%를 추출해서 합치기
even_data_df = even_data_df.sample(frac=0.25)
odd_data_df = odd_data_df.sample(frac=0.25)

data_df = pd.concat([even_data_df, odd_data_df])

len(data_df)

10328

In [70]:
data_df.reset_index(drop=True, inplace=True)

In [71]:
data_df_10328_1_1 = data_df

In [72]:
# 저장
data_df_10328_1_1.to_csv('../data/train/session1_session2_1_1.csv', index=False)

In [15]:
input = eval(data_df['session_dialog'].tolist()[0])
input

['반가워요~~ 저는 20대 여자입니다. 저는 아메리카노를 자주 마셔요.',
 '오 일 하시는구나. 저는 뉴질랜드로 어학연수를 가요.',
 '지금 일단 계획은 2년이요.',
 '아 해외 나가는 거 별로 안 좋아하세요?',
 '아 그렇군요. 그렇다면 고민되겠네요..',
 '아 그런가요? 거기는 회의나 그런건 자주 안 하나봐요.',
 '좋은데요? 나름 매력있는 직업같네요.',
 '네 좋네요. 그럼 저는 어학연수때 필요한 것 좀 준비하러 가볼게요.']

In [30]:
def gen(style, text):
    gened = model.generate(
        **tokenizer(
            f"""### 원문: {text}\n### 문체: {style}\n### 수정문:""",
            return_tensors='pt',
            return_token_type_ids=False
        ).to("cuda"),
        max_new_tokens=100,
        temperature=1,
        do_sample=True,
        repetition_penalty=1.2,
        num_beams=3
    )

    result = tokenizer.decode(gened[0], skip_special_tokens=True)
    return result[result.find("수정문:")+5:].replace("<|endoftext|>","")

In [31]:
model=model.to('cuda')
model.eval()
outputs=[]
with torch.no_grad():
    for text in input:
        output = gen('반말체',text)
        print(output)
        outputs.append(output)

안녕, 나는 20대 여자야. 나는 아메리카노를 자주 마셔.
오, 일 하는 구나. 나는 뉴질랜드로 어학연수를 가.
지금 일단 계획은 2년이야.
아 해외 나가는 거 별로 안 좋아해?
그렇군. 그러면 고민이 될 것 같아.
아 그런거야? 거기는 회의나 그런건 자주 안 해?
좋은 것 같아. 나름 매력있는 직업같아.
네 좋아. 그럼 나는 어학연수때 필요한 것 좀 준비하러 갈게.


In [37]:
# data_df의 session_dialog를 반말체로 변환한 결과를 outputs에 저장하고, 이를 data_df에 추가한 뒤, csv 파일로 저장

session_dialogs = [eval(dialog) for dialog in data_df['session_dialog'].tolist()]

In [73]:
session_dialogs_10328 = [eval(dialog) for dialog in data_df_10328_1_1['session_dialog'].tolist()]

In [46]:
from tqdm import tqdm

In [74]:
total_banmal_dialogs = []

for session_dialog in tqdm(session_dialogs_10328):
    banmal_dialogs = []
    with torch.no_grad():
        for dialog in session_dialog:
            banmal_dialogs.append(gen('반말체',dialog))
    total_banmal_dialogs.append(banmal_dialogs)

100%|██████████| 10328/10328 [22:49:56<00:00,  7.96s/it]  


In [76]:
data_df_10328_1_1['banmal_dialog'] = total_banmal_dialogs

In [77]:
data_df_10328_1_1

,id,session_dialog,session_persona,banmal_dialog
0,K5-01680-CL72369-CP01894-12-02-S2.json,"['안녕하세요. 잘 지내셨죠? 멍냥이들은 어떄요?', '저는 체험 여행을 자주가서요...","['나는 체험 여행을 자주간다.', '나는 경주 전통체험 여행을 갈 예정이다.']","[안녕, 잘 지냈어? 멍냥이들은 어때?, 나는 체험 여행을 자주가서, 이번에 문화체..."
1,K2-09621-CL21688-CP20773-16-07-S2.json,"['반갑네요. 오늘은 다른 날과 달리 날씨가 화창하네요.', '저는 엄청 뚱뚱하거 ...","['오늘은 날씨가 화창하다.', '나는 엄청 뚱뚱하다.', '요즘 사람들은 키가 큰...","[안녕. 오늘은 다른 날과 달리 날씨가 화창하네., 나는 엄청 뚱뚱해. 다이어트 같..."
2,K2-16238-CL35285-CP53786-13-09-S2.json,"['안녕하세요 2일만이네요 잘 지내셨나요?', '지금 이렇게 갑자기 추워지니까 올 ...","['나는 갑자기 추워져서 여름 폭염이 생각난다', '나는 기후 이상 현상이 늘어난다...","[안녕, 2일만이네. 잘 지냈어?, 지금 이렇게 갑자기 추워지니까 올 여름에 폭염이..."
3,K2-26348-CL25715-CP34833-05-02-S2.json,"['안녕하세요~ 저는 회계사로 일하는 20대 여자입니다.', '반가워요. 직업이 어...","['나는 회계사이다. 나는 20대 여자이다.', '나는 달리기를 가끔 한다. 나는 ...","[안녕, 나는 회계사로 일하는 20대 여자야., 안녕. 직업이 뭐야?, 아하, 그러..."
4,K2-22456-CL02291-CP01392-17-07-S2.json,"['네. 저는 맛있게 먹었습니다. 저는 라멘을 먹었습니다! 메뉴가 뭐였나요?', '...","['나는 자장면이 싫다.', '나는 일식을 좋아한다.']","[응, 나는 맛있게 먹었어. 나는 라멘을 먹었어! 메뉴가 뭐였어?, 상사와 입맛이 ..."
...,...,...,...,...
10323,K2-01208-CL20656-CP21290-14-12-S2.json,"['랜챗 매칭되신 걸 환영해요! 저도 20대 남자예요. 반가워요!', '저는 에세이...","['나는 20대 남자다.', '나는 에세이를 좋아한다.', '나는 인류학 부전공이다...","[랜챗 매칭되신 걸 환영해! 나도 20대 남자야. 반가워!, 나는 에세이 좋아해서 ..."
10324,K2-00161-CL13219-CP20829-19-03-S2.json,"['안녕하세요. 저는 10대 여성입니다.', '저는 요즘 매일 자전거로 학교에 가고...","['나는 10대 여성이다.', '나는 매일 자전거로 학교에 가고 있다.', '나는 ...","[안녕, 나는 10대 여성이야., 나는 요즘 매일 자전거로 학교에 가고 있어. 선생..."
10325,K2-34368-CL13202-CP61629-09-10-S2.json,"['안녕하세요. 저는 10대 여성입니다.', '저는 요즘 아버지와 진학 문제로 갈등...","['나는 10대 여성이다', '나는 요즘 진학 문제로 아버지와 갈등이 있다', '나...","[안녕, 나는 10대 여성이야., 나는 요즘 아버지와 진학 문제로 갈등이 있어., ..."
10326,K2-15070-CL21845-CP26190-08-14-S2.json,['반갑습니다~ 저도 20대 여성이에요 저는 친구가 많지 않아서 친구를 사귀고 싶어...,"['나는 20대 여성이다.', '나는 친구가 많지 않아 친구를 사귀고 싶다.', '...","[안녕, 나도 20대 여성이야. 나는 친구가 많지 않아서 친구를 사귀고 싶어., 그..."


In [78]:
data_df_10328_1_1.to_csv('../data/train/banmal_session12_10328.csv', index=False)

In [79]:
# train, val을 8대 2로 나누기
import pandas as pd

from sklearn.model_selection import train_test_split

train, validation = train_test_split(data_df_10328_1_1, test_size=0.2, random_state=42)

In [80]:
train.to_csv('../data/train/banmal_session12_10328_train.csv', index=False)
validation.to_csv('../data/val/banmal_session12_10328_val.csv', index=False)